In [1]:
from transformers import BertTokenizer, BertForPreTraining, BertModel
import torch
import Embedding_Model as em
from transformers import AdamW
import random

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yfr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yfr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yfr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
class MLMDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __getitem__(self, idx):
        input_ids = self.encodings['input_ids'][idx].clone()
        labels = input_ids.clone()
        
        # Create a mask for 15% of the tokens (as in your code)
        mask_arr = (torch.rand(input_ids.shape) < 0.15) & (input_ids != tokenizer.pad_token_id)
        masked_indices = mask_arr.nonzero()
        
        # Mask 15% of selected tokens
        input_ids[masked_indices] = tokenizer.mask_token_id
        
        # Prepare the labels (i.e., the original unmasked tokens)
        labels[~mask_arr] = -100  # -100 is a special value for masked tokens
        
        return {
            'input_ids': input_ids,
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': labels
        }
    
    def __len__(self):
        return len(self.encodings.input_ids)


In [ ]:
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = OurDataset(inputs)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)